In [1]:
import os
import numpy as np
import time
import copy
import sys

ang_2_bohr = 1.0/0.52917721067

import cp2k_utilities as cu

In [2]:
folder = "/home/kristjan/sshfs/marconi_scratch/cp2k_si_morb/"

file_basis_set = folder + "BR"

file_xyz = folder + "Si.xyz"

file_molog = folder + "PROJ-COEFF-1_0.MOLog"

file_cp2k_inp = folder + "cp2k.inp"

In [3]:
time1 = time.time()
elem_basis_names, cell = cu.read_cp2k_input(file_cp2k_inp)
print("Read cp2k input: %.3f" % (time.time()-time1))

time1 = time.time()
at_positions, at_elems = cu.read_atoms(file_xyz)
print("Read xyz: %.3f" % (time.time()-time1))

time1 = time.time()
basis_sets = cu.read_basis_functions(file_basis_set, elem_basis_names)
print("Read basis sets: %.3f" % (time.time()-time1))

time1 = time.time()
morb_composition, morb_energies = cu.read_and_process_molog(file_molog)
print("Read molog: %.3f" % (time.time()-time1))

Read cp2k input: 0.175
Read xyz: 0.143
Read basis sets: 0.346
Reading CP2K MOs from:/home/kristjan/sshfs/marconi_scratch/cp2k_si_morb/PROJ-COEFF-1_0.MOLog
Band gap system
Found 21 MOs spanned by 896 basis functions centered on 64 atoms.
Read molog: 0.997


In [4]:
def select_morbs(morb_composition, morb_energies, emin, emax):
    sel_morb_composition = []
    sel_morb_energies = []
    for i_mo, morb in enumerate(morb_composition):
        energy = morb_energies[i_mo]
        if energy > emin and energy < emax:
            sel_morb_energies.append(energy)
            sel_morb_composition.append(morb)

    print("Selected mol. orbitals %d/%d" % (len(sel_morb_composition), len(morb_composition)))
    return sel_morb_composition, np.array(sel_morb_energies)

In [15]:
# Main calculation

# NB: only orthonormal cells are supported

pbc_edge = 3.0
pbc_edge *= ang_2_bohr

step = 0.08
step *= ang_2_bohr
cell_n = (np.round(cell/step)).astype(int)

def grid_morbital():
    
    ### Set up grids
    grid_time = time.time()
    
    dv = cell/cell_n
    vol_elem = dv[0]*dv[1]*dv[2]
    
    x_arr = np.arange(0, cell_n[0], 1) * dv[0]
    y_arr = np.arange(0, cell_n[1], 1) * dv[1]
    z_arr = np.arange(0, cell_n[2], 1) * dv[2]
    r_grid = np.meshgrid(x_arr, y_arr, z_arr, indexing='ij')
    
    extra_elems = (pbc_edge/dv).astype(int)
    
    big_cell_n = cell_n + 2*extra_elems
    big_cell = big_cell_n * dv
    
    x_arr_big = np.arange(-extra_elems[0], cell_n[0]+extra_elems[0], 1) * dv[0]
    y_arr_big = np.arange(-extra_elems[1], cell_n[1]+extra_elems[1], 1) * dv[1]
    z_arr_big = np.arange(-extra_elems[2], cell_n[2]+extra_elems[2], 1) * dv[2]
    r_grid_big = np.meshgrid(x_arr_big, y_arr_big, z_arr_big, indexing='ij')    

    print("----Setting up grids %.4f s" % (time.time()-grid_time))
    ### -------------------------------------------------------------------------------
    ### Calculate atomic and molecular orbitals on the grid
    morb_time = time.time()
    
    # one data set on the grid for every molecular orbital
    morbs = [np.zeros(cell_n) for _ in range(len(morb_composition))]
    
    aorbs = []
    
    for i_at in range(len(at_positions)):
        elem = at_elems[i_at][0]
        pos = at_positions[i_at]
        
        x_grid_rel_big = r_grid_big[0] - pos[0]
        y_grid_rel_big = r_grid_big[1] - pos[1]
        z_grid_rel_big = r_grid_big[2] - pos[2]
        
        r_vec_2 = x_grid_rel_big**2 + y_grid_rel_big**2 + z_grid_rel_big**2
        
        aorbs.append([])
        
        for i_shell, shell in enumerate(basis_sets[elem]):
            l = shell[0]
            es = shell[1]
            cs = shell[2]
            
            # Calculate the radial part of the atomic orbital
            radial_part = np.zeros(big_cell_n)
            for e, c in zip(es, cs):
                radial_part += c*np.exp(-1.0*e*r_vec_2)
            
            aorbs[-1].append([])
            
            for i, m in enumerate(range(-l, l+1, 1)):
                atomic_orb_big = radial_part*cu.spherical_harmonic_grid(l, m,
                                                                     x_grid_rel_big,
                                                                     y_grid_rel_big,
                                                                     z_grid_rel_big)
                
                ### ---------------------------------------------------------------------------
                ### Reduce the periodic edges to the main cell
               
                # How many times the direction has to be "folded"
                n_fold = pbc_edge/cell
                ee = extra_elems
                
                for ix in range(int(np.ceil(n_fold[0]))):
                    imag_s1 = ee[0] + (ix+1)*cell_n[0]
                    imag_e1 = imag_s1 + cell_n[0]
                    
                    imag_s2 = ee[0] - (ix+1)*cell_n[0]
                    imag_e2 = imag_s2 + cell_n[0]
                    
                    if imag_e1 > big_cell_n[0]:
                        end_diff = imag_e1 - big_cell_n[0]
                        atomic_orb_big[ee[0]:-ee[0]-end_diff, :, :] += atomic_orb_big[imag_s1:, :, :]
                        atomic_orb_big[ee[0]+end_diff:-ee[0], :, :] += atomic_orb_big[:imag_e2, :, :]
                    else:
                        atomic_orb_big[ee[0]:-ee[0], :, :] += atomic_orb_big[imag_s1:imag_e1, :, :]
                        atomic_orb_big[ee[0]:-ee[0], :, :] += atomic_orb_big[imag_s2:imag_e2, :, :]
                       
                for iy in range(int(np.ceil(n_fold[1]))):
                    imag_s1 = ee[1] + (iy+1)*cell_n[1]
                    imag_e1 = imag_s1 + cell_n[1]
                    
                    imag_s2 = ee[1] - (iy+1)*cell_n[1]
                    imag_e2 = imag_s2 + cell_n[1]
                    
                    if imag_e1 > big_cell_n[1]:
                        end_diff = imag_e1 - big_cell_n[1]
                        atomic_orb_big[:, ee[1]:-ee[1]-end_diff, :] += atomic_orb_big[:, imag_s1:, :]
                        atomic_orb_big[:, ee[1]+end_diff:-ee[1], :] += atomic_orb_big[:, :imag_e2, :]
                    else:
                        atomic_orb_big[:, ee[1]:-ee[1], :] += atomic_orb_big[:, imag_s1:imag_e1, :]
                        atomic_orb_big[:, ee[1]:-ee[1], :] += atomic_orb_big[:, imag_s2:imag_e2, :]
                       
                for iz in range(int(np.ceil(n_fold[2]))):
                    imag_s1 = ee[2] + (iz+1)*cell_n[2]
                    imag_e1 = imag_s1 + cell_n[2]
                    
                    imag_s2 = ee[2] - (iz+1)*cell_n[2]
                    imag_e2 = imag_s2 + cell_n[2]
                    
                    if imag_e1 > big_cell_n[2]:
                        end_diff = imag_e1 - big_cell_n[2]
                        atomic_orb_big[:, :, ee[2]:-ee[2]-end_diff] += atomic_orb_big[:, :, imag_s1:]
                        atomic_orb_big[:, :, ee[2]+end_diff:-ee[2]] += atomic_orb_big[:, :, :imag_e2]
                    else:
                        atomic_orb_big[:, :, ee[2]:-ee[2]] += atomic_orb_big[:, :, imag_s1:imag_e1]
                        atomic_orb_big[:, :, ee[2]:-ee[2]] += atomic_orb_big[:, :, imag_s2:imag_e2]
                ### ---------------------------------------------------------------------------
                
                atomic_orb = atomic_orb_big[ee[0]:-ee[0],
                                            ee[1]:-ee[1],
                                            ee[2]:-ee[2]]
                
                #if i_at == 0 and i_shell == 0 and i == 0:
                #    write_cube_file("aorb.cube", file_xyz, cell, cell_n, atomic_orb)
                
                #aorbs[-1][-1].append(atomic_orb)
                # Add the atomic orbital with the correct coefficient to the molecular orbitals
                for i_mo in range(len(morb_composition)):
                    coef = morb_composition[i_mo][i_at][i_shell][i]
                    morbs[i_mo] += coef*atomic_orb
                
    print("----Putting molecular orbs on grid %.4f s" % (time.time()-morb_time))
    return morbs, morb_energies#, aorbs


In [16]:
morbs_on_grid, morb_energies = grid_morbital()

----Setting up grids 0.2768 s
----Putting molecular orbs on grid 960.2113 s


In [17]:
morb_energies

array([ -1.19044689e+00,  -1.19044689e+00,  -1.19044689e+00,
        -1.19044689e+00,  -1.19044689e+00,  -1.19044689e+00,
        -1.19044689e+00,  -1.19044689e+00,  -3.05266923e-12,
        -1.59872116e-14,   0.00000000e+00,   7.54809416e-01,
         7.54809416e-01,   7.54809416e-01,   7.54811539e-01,
         7.54811539e-01,   7.54811539e-01,   1.17040488e+00,
         1.17040489e+00,   1.17040489e+00,   1.17040489e+00])

In [18]:
for i in [9, 10, 11]:
    fname = "/home/kristjan/local_work/cubes/mo%d.cube" % i
    cu.write_cube_file(fname, file_xyz, cell, cell_n, morbs_on_grid[i])

In [22]:
# check for normalization
dv = cell/cell_n
vol_elem = dv[0]*dv[1]*dv[2]
np.sum(morbs_on_grid[8]**2)*vol_elem

1.0000174661111054

In [14]:
dv = cell/cell_n
vol_elem = dv[0]*dv[1]*dv[2]

# Calculate overlap between two orbitals
# vol_elem in bohr!
def overlap(data1, data2, vol_elem):
    return np.abs(np.dot(data1.flatten(), data2.flatten())*vol_elem)

print("Molecular orbital overlap matrix")
for i in range(len(morbs_on_grid)):
    print("%10d " %i, end='')
print()
for i in range(len(morbs_on_grid)):
    for j in range(len(morbs_on_grid)):
        ol = overlap(morbs_on_grid[i], morbs_on_grid[j], vol_elem)
        print("%10.4f " % ol, end='')
    print()
    

Molecular orbital overlap matrix
         0          1          2          3          4          5          6          7          8          9         10         11         12         13         14         15         16         17         18         19         20 
    0.9924     0.0000     0.0002     0.0000     0.0000     0.0000     0.0320     0.0209     0.0000     0.0003     0.0000     0.0001     0.0000     0.0000     0.0000     0.0000     0.0000     0.0000     0.0000     0.0000     0.0000 
    0.0000     0.9924     0.0000     0.0002     0.0000     0.0000     0.0209     0.0320     0.0000     0.0000     0.0003     0.0000     0.0001     0.0000     0.0000     0.0000     0.0000     0.0000     0.0000     0.0000     0.0000 
    0.0002     0.0000     1.0563     0.0000     0.0000     0.0000     0.0158     0.0103     0.0000     0.0000     0.0000     0.0001     0.0000     0.0000     0.0000     0.0000     0.0000     0.0000     0.0000     0.0025     0.0000 
    0.0000     0.0002     0.0000     1.

In [18]:
for i_at in range(len(aorbs_on_grid)):
    for i_nr in range(len(aorbs_on_grid[i_at])):
        for i_m in range(len(aorbs_on_grid[i_at][i_nr])):
            fname = "ao%d_%d_%d.cube" % (i_at, i_nr, i_m)
            write_cube_file(fname, file_xyz, cell, cell_n, aorbs_on_grid[i_at][i_nr][i_m])